In [90]:
import ee
from datetime import datetime

In [ ]:
ee.Authenticate()

In [5]:
ee.Initialize()

In [177]:
bands = ['B2', 'B3', 'B4', 'B8', 'pB2', 'pB3', 'pB4', 'pB8', 'mask']

coords = [-44.51335,-5.81875]
point = ee.Geometry.Point(coords)

sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR')
sentinel2 = sentinel2.filterBounds(point)

In [174]:
# Alerts mapbiomas
alerts_mapbiomas_all = ee.FeatureCollection('users/valderli/mapbiomas_alerts_cena_23mnp')

date_start = datetime.strptime('01/01/2019', '%d/%m/%Y').timestamp() * 1000
#date_end = datetime.strptime('01/01/2019', '%d/%m/%Y').timestamp() * 1000

alerts_mapbiomas = alerts_mapbiomas_all.filterMetadata('beforeDate', 'greater_than', date_start)
#alerts_mapbiomas = alerts_mapbiomas.filterMetadata('afterDate', 'less_than', date_end);

alerts_mapbiomas_features = alerts_mapbiomas.getInfo()['features']

In [179]:
# Define function of buffering and bounding      
def bounding_box(feature):
    buffer = feature.buffer(500)  # buffer in m
    box = buffer.bounds()
    return(box)

alerts_mapbiomas_boxes = alerts_mapbiomas.map(bounding_box);
alerts_mapbiomas_boxes_features = alerts_mapbiomas_boxes.getInfo()['features']

In [79]:
def maskS2clouds(image):
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) and (qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask);

In [92]:
def millisecondsTodate(m_seconds):
    return datetime.fromtimestamp(m_seconds / 1000.0).strftime('%Y-%m-%d')

In [187]:
def exportToCloudStorage(image, name):
    task = ee.batch.Export.image.toCloudStorage(
        image = image.select(bands).toInt(),
        description = name,
        bucket = 'tensorflow-fire-cerrado1',
        fileNamePrefix = 'images_train_test_alerts_deforestation/{0}'.format(name),
        maxPixels = 1e13,
        scale = 10,
        region = None
    );
    task.start()
    
def exportToDrive(image, name):
    task = ee.batch.Export.image.toDrive(
        image = image.select(bands).toInt(),
        folder='images_train_test_alerts_deforestation',
        description = name,
        maxPixels = 1e13,
        scale = 10,
        region = None
    );
    task.start()

In [ ]:
# um dia em milissegundos
m_day = 8.64e+7

for index, alert_box in enumerate(alerts_mapbiomas_boxes_features):

    #if index == 17:
    props = alert_box['properties']

    before_date = props['beforeDate']
    after_date  = props['afterDate']
    alert_id    = props['IDAlerta']

    region = ee.Feature(alert_box)

    # image before alert
    before_end = before_date - 5 * m_day
    before_start = before_end - 15 * m_day

    before_date_start = millisecondsTodate(before_start)
    before_date_end = millisecondsTodate(before_end)

    before_image = sentinel2.filterDate(before_date_start, before_date_end)
    before_image = before_image.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    before_image = before_image.map(maskS2clouds).median()

    # image after alert
    after_start = after_date + 2 * m_day
    after_end = after_start + 15 * m_day

    after_date_start = millisecondsTodate(after_start)
    after_date_end = millisecondsTodate(after_end)

    after_image = sentinel2.filterDate(after_date_start, after_date_end)
    after_image = after_image.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    after_image = after_image.map(maskS2clouds).median()

    # merge images before and after alert
    image = before_image.addBands(after_image.select('B2').rename('pB2'))
    image = image.addBands(after_image.select('B3').rename('pB3'))
    image = image.addBands(after_image.select('B4').rename('pB4'))
    image = image.addBands(after_image.select('B8').rename('pB8'));

    # mask
    alert = alerts_mapbiomas_features[index]
    feature_alert = ee.Feature(alert)
    mask = ee.Image(1).clip(feature_alert).unmask(ee.Image.constant(0));
    mask = mask.select(['constant'], ['mask'])

    image = image.addBands(mask.select('mask'));

    # clip image
    image = image.select(bands).clip(region)

    # export image
    exportToDrive(image, 'alert_id_{0}'.format(alert_id))

    print("id {0}: {1}".format(alert_id, millisecondsTodate(before_date)))